# Multi-channel Delaunay Registration

In [1]:
#import libraries

import tifffile as tiff
import cv2
import numpy as np
import time
import os
from cpselect.cpselect import cpselect

Configure the settings below. 

*The tif files don't need to have the same dimensions or resolution. Additionally, the script has been tested on both 8bit & 16bit files.

*The ref_chnl params indicate which slice in each tif stack/image to use for the moving and target image. Since live is typically a single frame, tar_ref_chnl = 0. In this example, the 5ch_hcr file contains the mCherry reference in the 2nd slice, so mov_ref_chnl = 1.

*(These are the only parameters you need to change)*

In [2]:
# Define the file paths
params = {
    'output_dir': r"C:\Users\emricklab\Desktop",
    'output_filename': r"stack.tiff",
    'moving': r"C:\Users\emricklab\Desktop\Hcr_5ch_crop.tif",
    'mov_ref_chnl': 1,
    'target': r"C:\Users\emricklab\Desktop\Live.tif",
    'tar_ref_chnl': 0  # 
}

Import the target & reference mCherry.

In [3]:
# Reading the moving stack
moving = []
ret, moving = cv2.imreadmulti(mats=moving, filename=params['moving'], flags=cv2.IMREAD_COLOR)

# Reading the target image
target = []
ret2, target = cv2.imreadmulti(mats=target, filename=params['target'], flags=cv2.IMREAD_COLOR)

# Select Referance mCherry
mcherry_hcr = "mcherry_hcr.tif" 
cv2.imwrite(mcherry_hcr, moving[params['mov_ref_chnl']]) #channel 1 = 2nd slice of tif stack = mcherry ref

mcherry_live = "mcherry_live.tif" 
cv2.imwrite(mcherry_live, target[params['tar_ref_chnl']])
img2 = cv2.imread(mcherry_live)

Use the cpselect GUI to acquire point-correspondence landmarks. Refer to https://github.com/hofmann-tobias/cpselect for usage.

In [4]:
#select landmarks in mcherry_hcr and mcherry_live
controlpointlist = cpselect(mcherry_hcr, mcherry_live)

# Clean up the temporary filepaths
os.remove(mcherry_hcr)
os.remove(mcherry_live)

#convert controlpointlist to landmark lists for individual hcr/live mcherry
landmarks_points = []
landmarks_points2 = []

for point in controlpointlist:
    landmarks_points.append([point['img1_x'], point['img1_y']])
    landmarks_points2.append([point['img2_x'], point['img2_y']])


Create a rectangular bounding box for the pool of landmarks, then convert the landmark lists to int32 points to make calculations faster

In [5]:
# Find minimum and maximum x and y coordinates
min_x = min(coord[0] for coord in landmarks_points)
max_x = max(coord[0] for coord in landmarks_points)
min_y = min(coord[1] for coord in landmarks_points)
max_y = max(coord[1] for coord in landmarks_points)

# Create corners of the bounding rectangle
top_left = [min_x, max_y]
top_right = [max_x, max_y]
bottom_left = [min_x, min_y]
bottom_right = [max_x, min_y]

# Add corners to the original list of coordinates
landmarks_points.append(top_left)
landmarks_points.append(top_right)
landmarks_points.append(bottom_left)
landmarks_points.append(bottom_right)

#REPEAT FOR LANDMARKS IN IMG2
min_x = min(coord[0] for coord in landmarks_points2)
max_x = max(coord[0] for coord in landmarks_points2)
min_y = min(coord[1] for coord in landmarks_points2)
max_y = max(coord[1] for coord in landmarks_points2)

top_left = [min_x, max_y]
top_right = [max_x, max_y]
bottom_left = [min_x, min_y]
bottom_right = [max_x, min_y]

landmarks_points2.append(top_left)
landmarks_points2.append(top_right)
landmarks_points2.append(bottom_left)
landmarks_points2.append(bottom_right)

Run the Delaunay registration algorithm

In [6]:
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

#IMG2 = FIXED
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

warped_images = []
#LOOP PROCESSING FOR EACH CHANNEL IN MOVING
for image in moving: 
    
    #GENERATE POINTS ARRAY FROM LANDMARK LIST
    points = np.array(landmarks_points, np.int32)
    points2 = np.array(landmarks_points2, np.int32)

    #GENERATE RECTANGULAR CONVEXHULLS
    convexhull = cv2.convexHull(points)
    convexhull2 = cv2.convexHull(points2)
    
    #Create empty image to place warp on
    height, width, channels = img2.shape
    img2_new_face = np.zeros((height, width, channels), np.uint8)
    
    #IMG 1 = MOVING
    img = image
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(img_gray)
    
    # Face 1
    cv2.fillConvexPoly(mask, convexhull, 255)
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)

    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])


        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)

        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)

        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)

        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)
            
    #FIX LINE ISSUE: mask the boundaries
    lines_space_mask = np.zeros_like(img_gray)
    lines_space_new_face = np.zeros_like(img2)
    
    
    # Triangulation of both faces
    for triangle_index in indexes_triangles:
        # Triangulation of the first face
        tr1_pt1 = landmarks_points[triangle_index[0]]
        tr1_pt2 = landmarks_points[triangle_index[1]]
        tr1_pt3 = landmarks_points[triangle_index[2]]
        triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)


        rect1 = cv2.boundingRect(triangle1)
        (x, y, w, h) = rect1
        cropped_triangle = img[y: y + h, x: x + w]
        cropped_tr1_mask = np.zeros((h, w), np.uint8)


        points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                           [tr1_pt2[0] - x, tr1_pt2[1] - y],
                           [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)

        cv2.fillConvexPoly(cropped_tr1_mask, points, 255)

        #convert floats to ints
        tr1_pt1 = [int(x) for x in tr1_pt1]
        tr1_pt2 = [int(x) for x in tr1_pt1]
        tr1_pt3 = [int(x) for x in tr1_pt1]

        # Lines space
        cv2.line(lines_space_mask, tr1_pt1, tr1_pt2, 255)
        cv2.line(lines_space_mask, tr1_pt2, tr1_pt3, 255)
        cv2.line(lines_space_mask, tr1_pt1, tr1_pt3, 255)
        lines_space = cv2.bitwise_and(img, img, mask=lines_space_mask)

        # Triangulation of second face
        tr2_pt1 = landmarks_points2[triangle_index[0]]
        tr2_pt2 = landmarks_points2[triangle_index[1]]
        tr2_pt3 = landmarks_points2[triangle_index[2]]
        triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)


        rect2 = cv2.boundingRect(triangle2)
        (x, y, w, h) = rect2

        cropped_tr2_mask = np.zeros((h, w), np.uint8)

        points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                            [tr2_pt2[0] - x, tr2_pt2[1] - y],
                            [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)

        cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)

        # Warp triangles
        points = np.float32(points)
        points2 = np.float32(points2)
        M = cv2.getAffineTransform(points, points2)
        warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

        # Reconstructing destination face
        img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
        img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
        _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
        warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

        img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
        img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area
        
    
    warped_images.append(img2_new_face)

    

Save the registered file to specified output dir

In [7]:
# Concatenate the images along the channel axis
stack = np.stack(warped_images, axis=0)

# Save the stack as a multi-channel TIFF
output_path = os.path.join(params['output_dir'], params['output_filename'])
tiff.imsave(output_path, stack, planarconfig='contig')

print("TIFF stack saved successfully.")

TIFF stack saved successfully.
